In [ ]:
import squidpy as sq
import spatialdata as sd
import pickle
import multimodal_cci as mmcci
import pickle
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import seaborn as sns
import copy
import spatialdata as sd
from matplotlib.colors import ListedColormap
from spatialdata import bounding_box_query
import spatialdata_plot

In [ ]:
# cosmx_zarr_path = "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Feng/clean-data/48974-2B_labeled_cosmx.zarr"
# cosmx_sdata = sd.read_zarr(cosmx_zarr_path)
# cosmx_sdata

In [ ]:
cell_type_colors = {
    "DC": "#5f9d9e",
    "Endothelial Cell": "#f8a41e",
    "Fibroblast": "#458b41",
    "KC Basal": "#f16b6b",
    "KC Cornified": "#9a1f61",
    "KC Granular": "#c72685",
    "KC Differentiating": "#9583bd",
    "KC Hair": "#eb2627",
    "LC": "#37479b",
    "Macrophage": "#eae71d",
    "Melanocytes": "#8b471f",
    "Melanoma": "black",
    "NK": "#99ca3e",
    "T Cell": "#41baeb",
    "Treg": "#bbe5f3",
    "pDC": "#66bf76",
    "Mast Cell": "#7f8133",
    "mRegDC":"#105146",
    'CD8+ T Cell': "#406573",
    "B Cell":"#fed9b9",
    "Pericytes":"#dca566",
    "Sweat gland related":"#f2634b",
    "nan":"grey",
    'Schwann Cell':'#0b507b',
   'Plasma' : '#f1ea9d',
   'KC IFN' : '#f06ba8', 
   "Monocytes":"#9cc7a1",
   "KC Dysplastic": "#d8c0dd",
   "Ambiguous":"grey",
}

In [ ]:
data_dir = "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/SkinCancerAtlas/cci_results/"
save_dir = "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Onkar_Levi/Individual_panels_fig9_10/Spatial_plots/"

# Read the stLearn CCI results
samples_single_spat = {
    'cosmx_mela': {
        '48974-2B': '48974-2B_cutoff50_cosmx_stlearn.pkl'},
    'cosmx_scc': {
        'B18_SCC':'B18_SCC_cutoff50_cosmx_stlearn.pkl'}
}

for tech, sample in samples_single_spat.items():
    for sample_name, file_name in samples_single_spat[tech].items():
        with open(data_dir + file_name, 'rb') as f:
            samples_single_spat[tech][sample_name] = pickle.load(f)

## Melanoma

In [ ]:
# samples_single_spat['cosmx_mela']['48974-2B'].obs['color'] = samples_single_spat['cosmx_mela']['48974-2B'].obs['cell_type'].map(color_map)
cell_types_to_keep = list(samples_single_spat['cosmx_mela']['48974-2B'].obs["cell_type"].unique())
subset_cell_type_colors = {key: cell_type_colors[key] for key in cell_types_to_keep if key in cell_type_colors}
subset_cell_type_colors = dict(sorted(subset_cell_type_colors.items()))

# Create a colormap from the dictionary
cell_types = list(subset_cell_type_colors.keys())
colors = [subset_cell_type_colors[cell] for cell in cell_types]
custom_cmap = ListedColormap(colors, name="custom_cell_type_cmap")

### FOV-12

In [ ]:
cosmx_zarr_path = "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Feng/clean-data/48974-2B_labeled_cosmx.zarr"
cosmx_sdata = sd.read_zarr(cosmx_zarr_path)

# Add the spatial information to uns slot
cosmx_sdata.table.uns['spatial'] = samples_single_spat['cosmx_mela']['48974-2B'].uns['spatial']
cosmx_sdata.table.obs = cosmx_sdata.table.obs.join(samples_single_spat['cosmx_mela']['48974-2B'].obs['cell_type'],how="outer")

crop0 = lambda x: bounding_box_query(
    x,
    min_coordinate=[0, 0],
    max_coordinate=[5000, 3500],
    axes=("x", "y"),
    target_coordinate_system="12",
)

In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="KIT"]
gene2 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="S100B"]
gene3 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="MKI67"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):
    
    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["12"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Get the current axes (for the spatial plot)
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    
    scatter_ax.scatter(gene1['x'], gene1['y'], color='black', marker='o', s=15, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='white', marker='o', s=15, zorder=10)
    scatter_ax.scatter(gene3['x'], gene3['y'], color='#FF2400', marker='o', s=15, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FOV12_KIT_S100B_MKI67.pdf", bbox_inches="tight")


#### IL34-CSF1R

In [ ]:
with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["12"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    plt.savefig(save_dir+"Mel_CosMx_FullView_FOV12_celltypes.pdf", bbox_inches="tight")


In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="IL34"]
gene2 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="CSF1R"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["12"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax)

    # Get the current axes (for the spatial plot)
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the scatter plot on the new axes
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_FOV12_IL34_CSF1R.pdf", bbox_inches="tight")


In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="IL34"]
gene2 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="CSF1R"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["12"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax  # Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the arrows and scatter points on scatter_ax
    threshold = 120  
    for i in range(len(gene1)):
        for j in range(len(gene2)):
            distance = np.sqrt((gene1['x'].iloc[i] - gene2['x'].iloc[j])**2 + 
                               (gene1['y'].iloc[i] - gene2['y'].iloc[j])**2)
            if distance < threshold:
                scatter_ax.arrow(gene1['x'].iloc[i], gene1['y'].iloc[i],
                                 gene2['x'].iloc[j] - gene1['x'].iloc[i],
                                 gene2['y'].iloc[j] - gene1['y'].iloc[i],
                                 color='white', alpha=1, width=2.5, head_width=10, length_includes_head=True,
                                 fill=True, linewidth=0.5, zorder=10)
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_FOV12_with_arrow_IL34_CSF1R.png", bbox_inches="tight")

#### S100A8-TLR4

In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="S100A8"]
gene2 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="TLR4"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["12"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax  # Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the arrows and scatter points on scatter_ax
    threshold = 120  
    for i in range(len(gene1)):
        for j in range(len(gene2)):
            distance = np.sqrt((gene1['x'].iloc[i] - gene2['x'].iloc[j])**2 + 
                               (gene1['y'].iloc[i] - gene2['y'].iloc[j])**2)
            if distance < threshold:
                scatter_ax.arrow(gene1['x'].iloc[i], gene1['y'].iloc[i],
                                 gene2['x'].iloc[j] - gene1['x'].iloc[i],
                                 gene2['y'].iloc[j] - gene1['y'].iloc[i],
                                 color='white', alpha=1, width=2.5, head_width=10, length_includes_head=True,
                                 fill=True, linewidth=0.5, zorder=10)
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_FOV12_with_arrow_S100A8_TLR4.png", bbox_inches="tight")

#### CXCL12-CXCR4

In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="CXCL12"]
gene2 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="CXCR4"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["12"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    

    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the scatter plot on the new axes
    scatter_ax.scatter(gene1['x'], gene1['y'], color='black', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='o', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_FOV12_CXCL12_CXCR4.png", bbox_inches="tight")


#### CCL19-CCR7

In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="CCL19"]
gene2 = crop0(cosmx_sdata).points['12_points'].compute().dropna()[crop0(cosmx_sdata).points['12_points'].compute()['target']=="CCR7"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["12"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the scatter plot on the new axes
    scatter_ax.scatter(gene1['x'], gene1['y'], color='black', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='o', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_FOV12_CCL19_CCR7.png", bbox_inches="tight")


### FOV-13

In [ ]:
cosmx_zarr_path = "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Feng/clean-data/48974-2B_labeled_cosmx.zarr"
cosmx_sdata = sd.read_zarr(cosmx_zarr_path)

# Add the spatial information to uns slot
cosmx_sdata.table.uns['spatial'] = samples_single_spat['cosmx_mela']['48974-2B'].uns['spatial']
cosmx_sdata.table.obs = cosmx_sdata.table.obs.join(samples_single_spat['cosmx_mela']['48974-2B'].obs['cell_type'],how="outer")

crop0 = lambda x: bounding_box_query(
    x,
    min_coordinate=[0, 0],
    max_coordinate=[5000, 3500],
    axes=("x", "y"),
    target_coordinate_system="13",
)

#### IL34-CSF1R

In [ ]:
with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    plt.savefig(save_dir+"Mel_CosMx_FullView_celltypes.pdf", bbox_inches="tight")


In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="IL34"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="CSF1R"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Get the current axes (for the spatial plot)
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    # Create a new set of axes for the scatter plot so it doesn't affect the spatial plot
    scatter_ax = ax.twinx()  # Creates a new y-axis on the right side (doesn't affect the original y-axis)
    
    # Plot the scatter plot on the new axes
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_IL34_CSF1R.pdf", bbox_inches="tight")


In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="IL34"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="CSF1R"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax  # Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the arrows and scatter points on scatter_ax
    threshold = 120  
    for i in range(len(gene1)):
        for j in range(len(gene2)):
            distance = np.sqrt((gene1['x'].iloc[i] - gene2['x'].iloc[j])**2 + 
                               (gene1['y'].iloc[i] - gene2['y'].iloc[j])**2)
            if distance < threshold:
                scatter_ax.arrow(gene1['x'].iloc[i], gene1['y'].iloc[i],
                                 gene2['x'].iloc[j] - gene1['x'].iloc[i],
                                 gene2['y'].iloc[j] - gene1['y'].iloc[i],
                                 color='white', alpha=1, width=2.5, head_width=10, length_includes_head=True,
                                 fill=True, linewidth=0.5, zorder=10)
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_with_arrow_IL34_CSF1R.png", bbox_inches="tight")

### S100A8-TLR4

In [ ]:
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="S100A8"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="TLR4"]

#======================================================================================================================================================================================    
with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Get the current axes (for the spatial plot)
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    # Create a new set of axes for the scatter plot so it doesn't affect the spatial plot
    scatter_ax = ax.twinx()  # Creates a new y-axis on the right side (doesn't affect the original y-axis)
    
    # Plot the scatter plot on the new axes
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_S100A8_TLR4.pdf", bbox_inches="tight")


In [ ]:
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="S100A8"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="TLR4"]

#======================================================================================================================================================================================    
with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax  # Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the arrows and scatter points on scatter_ax
    threshold = 120  
    for i in range(len(gene1)):
        for j in range(len(gene2)):
            distance = np.sqrt((gene1['x'].iloc[i] - gene2['x'].iloc[j])**2 + 
                               (gene1['y'].iloc[i] - gene2['y'].iloc[j])**2)
            if distance < threshold:
                scatter_ax.arrow(gene1['x'].iloc[i], gene1['y'].iloc[i],
                                 gene2['x'].iloc[j] - gene1['x'].iloc[i],
                                 gene2['y'].iloc[j] - gene1['y'].iloc[i],
                                 color='white', alpha=1, width=2.5, head_width=10, length_includes_head=True,
                                 fill=True, linewidth=0.5, zorder=10)
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_with_arrow_S100A8_TLR4.png", bbox_inches="tight")

### Zoomed in other LR pairs - Visium and Xenium validation

In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="CCL19"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="CCR7"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)


    plt.savefig(save_dir+"Mel_CosMx_FullView_CCL19_CCR7.png", bbox_inches="tight")


In [ ]:
#======================================================================================================================================================================================    
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="CXCL12"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="CXCR4"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_mela']['48974-2B'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"Mel_CosMx_FullView_CXCL12_CXCR4.png", bbox_inches="tight")

## SCC

### FOV-14 (has no DCs; custom color map) 

#### SOX2-LAMP3

In [ ]:
cosmx_zarr_path = "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Feng/clean-data/B18_SCC_labeled_cosmx.zarr"
cosmx_sdata = sd.read_zarr(cosmx_zarr_path)

# Add the spatial information to uns slot
cosmx_sdata.table.uns['spatial'] = samples_single_spat['cosmx_scc']['B18_SCC'].uns['spatial']
cosmx_sdata.table.obs = cosmx_sdata.table.obs.join(samples_single_spat['cosmx_scc']['B18_SCC'].obs['cell_type'],how="outer")

crop0 = lambda x: bounding_box_query(
    x,
    min_coordinate=[0, 0],
    max_coordinate=[5000, 3500],
    axes=("x", "y"),
    target_coordinate_system="14",
)

########################################################################################################################################################################

cell_types_to_keep = set(samples_single_spat['cosmx_scc']['B18_SCC'].obs[samples_single_spat['cosmx_scc']['B18_SCC'].obs["fov"]=="14"]["cell_type"]) #list(samples_single_spat['cosmx_scc']['B18_SCC'].obs["cell_type"].unique())
subset_cell_type_colors = {key: cell_type_colors[key] for key in cell_types_to_keep if key in cell_type_colors}
subset_cell_type_colors = dict(sorted(subset_cell_type_colors.items()))

# Create a colormap from the dictionary
cell_types = list(subset_cell_type_colors.keys())
colors = [subset_cell_type_colors[cell] for cell in cell_types]
custom_cmap = ListedColormap(colors, name="custom_cell_type_cmap")

In [ ]:
with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_scc']['B18_SCC'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["14"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    plt.savefig(save_dir+"SCC-BCC_CosMx_FullView_celltypes.pdf", bbox_inches="tight")


In [ ]:
gene1 = crop0(cosmx_sdata).points['14_points'].compute().dropna()[crop0(cosmx_sdata).points['14_points'].compute()['target']=="SOX2"]
gene2 = crop0(cosmx_sdata).points['14_points'].compute().dropna()[crop0(cosmx_sdata).points['14_points'].compute()['target']=="LAMP3"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
    samples_single_spat['cosmx_scc']['B18_SCC'],
    color="cell_type",  # Color by the 'cell_type' column
    library_key="fov",
    library_id=["14"],
    seg_cell_id="cell_ID",
    seg_outline=True,
    img=False,
    figsize=(15, 10),
    scalebar_dx=1.0,
    scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
    palette=custom_cmap,
    ax=ax# Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    
    scatter_ax.scatter(gene1['x'], gene1['y'], color='black', marker='o', s=15, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='white', marker='o', s=15, zorder=10)


    plt.savefig(save_dir+"SCC-BCC_CosMx_FullView_SOX2_LAMP3_fov14.pdf", bbox_inches="tight")

## SCC - arrow plots (all cell-types)

In [ ]:
cell_types_to_keep = list(samples_single_spat['cosmx_scc']['B18_SCC'].obs["cell_type"].unique())
subset_cell_type_colors = {key: cell_type_colors[key] for key in cell_types_to_keep if key in cell_type_colors}
subset_cell_type_colors = dict(sorted(subset_cell_type_colors.items()))

# Create a colormap from the dictionary
cell_types = list(subset_cell_type_colors.keys())
colors = [subset_cell_type_colors[cell] for cell in cell_types]
custom_cmap = ListedColormap(colors, name="custom_cell_type_cmap")

########################################################################################################################################################################

cosmx_zarr_path = "/QRISdata/Q2051/SCC_Paper/resources/data/Data_For_Github_Codes/Feng/clean-data/B18_SCC_labeled_cosmx.zarr"
cosmx_sdata = sd.read_zarr(cosmx_zarr_path)

# Add the spatial information to uns slot
cosmx_sdata.table.uns['spatial'] = samples_single_spat['cosmx_scc']['B18_SCC'].uns['spatial']
cosmx_sdata.table.obs = cosmx_sdata.table.obs.join(samples_single_spat['cosmx_scc']['B18_SCC'].obs['cell_type'],how="outer")

crop0 = lambda x: bounding_box_query(
    x,
    min_coordinate=[0, 0],
    max_coordinate=[5000, 3500],
    axes=("x", "y"),
    target_coordinate_system="14",
)

### IL34-CSF1R

In [ ]:
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="IL34"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="CSF1R"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_scc']['B18_SCC'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Get the current axes (for the spatial plot)
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    # Create a new set of axes for the scatter plot so it doesn't affect the spatial plot
    scatter_ax = ax.twinx()  # Creates a new y-axis on the right side (doesn't affect the original y-axis)
    
    # Plot the scatter plot on the new axes
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"SCC-BCC_CosMx_FullView_IL34_CSF1R.pdf", bbox_inches="tight")


In [ ]:
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="IL34"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="CSF1R"]

#======================================================================================================================================================================================    
with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_scc']['B18_SCC'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the arrows and scatter points on scatter_ax
    threshold = 120  
    for i in range(len(gene1)):
        for j in range(len(gene2)):
            distance = np.sqrt((gene1['x'].iloc[i] - gene2['x'].iloc[j])**2 + 
                               (gene1['y'].iloc[i] - gene2['y'].iloc[j])**2)
            if distance < threshold:
                scatter_ax.arrow(gene1['x'].iloc[i], gene1['y'].iloc[i],
                                 gene2['x'].iloc[j] - gene1['x'].iloc[i],
                                 gene2['y'].iloc[j] - gene1['y'].iloc[i],
                                 color='white', alpha=1, width=2.5, head_width=10, length_includes_head=True,
                                 fill=True, linewidth=0.5, zorder=10)
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"SCC-BCC_CosMx_FullView_with_arrow_IL34_CSF1R.png", bbox_inches="tight")

### S100A8-TLR4

In [ ]:
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="S100A8"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="TLR4"]

with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (300)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_scc']['B18_SCC'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Get the current axes (for the spatial plot)
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    # Create a new set of axes for the scatter plot so it doesn't affect the spatial plot
    scatter_ax = ax.twinx()  # Creates a new y-axis on the right side (doesn't affect the original y-axis)
    
    # Plot the scatter plot on the new axes
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"SCC-BCC_CosMx_FullView_S100A8_TLR4.pdf", bbox_inches="tight")


In [ ]:
gene1 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="S100A8"]
gene2 = crop0(cosmx_sdata).points['13_points'].compute().dropna()[crop0(cosmx_sdata).points['13_points'].compute()['target']=="TLR4"]

#======================================================================================================================================================================================    
with plt.rc_context({"figure.figsize": (5, 4), "figure.dpi": (600)}):

    fig, ax = plt.subplots(figsize=(15, 10))
    # Plot the spatial segment plot first
    sq.pl.spatial_segment(
        samples_single_spat['cosmx_scc']['B18_SCC'],
        color="cell_type",  # Color by the 'cell_type' column
        library_key="fov",
        library_id=["13"],
        seg_cell_id="cell_ID",
        seg_outline=True,
        img=False,
        figsize=(15, 10),
        scalebar_dx=1.0,
        scalebar_kwargs={"scale_loc": "bottom", "location": "lower right"},
        palette=custom_cmap,
        ax=ax# Pass the custom colormap here
    )
    
    # Sync the aspect ratio and limits of scatter_ax with ax
    ax.set_aspect('equal', adjustable='datalim')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Create a new y-axis on the right side
    scatter_ax = ax.twinx()
    scatter_ax.set_ylim(ylim[::-1])
    
    # Plot the arrows and scatter points on scatter_ax
    threshold = 120  
    for i in range(len(gene1)):
        for j in range(len(gene2)):
            distance = np.sqrt((gene1['x'].iloc[i] - gene2['x'].iloc[j])**2 + 
                               (gene1['y'].iloc[i] - gene2['y'].iloc[j])**2)
            if distance < threshold:
                scatter_ax.arrow(gene1['x'].iloc[i], gene1['y'].iloc[i],
                                 gene2['x'].iloc[j] - gene1['x'].iloc[i],
                                 gene2['y'].iloc[j] - gene1['y'].iloc[i],
                                 color='white', alpha=1, width=2.5, head_width=10, length_includes_head=True,
                                 fill=True, linewidth=0.5, zorder=10)
    scatter_ax.scatter(gene1['x'], gene1['y'], color='pink', marker='o', s=3, zorder=10)
    scatter_ax.scatter(gene2['x'], gene2['y'], color='red', marker='^', s=3, zorder=10)

    plt.savefig(save_dir+"SCC-BCC_CosMx_FullView_with_arrow_S100A8_TLR4.png", bbox_inches="tight")